# Exploratory Data Analysis: Crop Production Trends
This notebook explores crop production trends over the years, focusing on the **highest production crop (Tame Hay)** and the **lowest production crop (Peas, Dry)**. The analysis includes:
- General overview of the dataset
- Trends in crop production over the years
- Comparison of Tame Hay and Peas, Dry
- Insights into production variability and country-level contributions
- Correlation between production and farm value

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
sns.set(style="whitegrid")

## 1. Load and Inspect the Dataset
First, we load the dataset and perform basic checks to understand its structure and contents.

In [ ]:
df = pd.read_csv('farm_production_dataset.csv')
print("Dataset Overview:")
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10273 entries, 0 to 10272
Data columns (total 9 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   REF_DATE                                10273 non-null  int64  
 1   GEO                                     10273 non-null  object 
 2   Type of crop                            10272 non-null  object 
 3   Average farm price (dollars per tonne)  10243 non-null  float64
 4   Average yield (kilograms per hectare)   10246 non-null  float64
 5   Production (metric tonnes)              10245 non-null  float64
 6   Seeded area (acres)                     9873 non-null   float64
 7   Seeded area (hectares)                  9847 non-null   float64
 8   Total farm value (dollars)              10273 non-null  int64  
dtypes: float64(5), int64(2), object(2)
memory usage: 722.4+ KB


,REF_DATE,Average farm price (dollars per tonne),Average yield (kilograms per hectare),Production (metric tonnes),Seeded area (acres),Seeded area (hectares),Total farm value (dollars)
count,10273.000000,10243.000000,10246.000000,1.024500e+04,9.873000e+03,9.847000e+03,1.027300e+04
mean,1947.661053,59.633078,3647.442319,1.010887e+06,1.310067e+06,5.316231e+05,5.490084e+04
std,22.204519,90.920549,8068.854966,3.044681e+06,4.881387e+06,1.978043e+06,2.502413e+05
min,1908.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1929.000000,21.000000,1055.000000,8.300000e+03,1.050000e+04,4.350000e+03,3.690000e+02
50%,1948.000000,43.000000,1485.000000,7.195000e+04,8.300000e+04,3.400000e+04,3.006000e+03
75%,1967.000000,76.000000,2363.750000,5.480000e+05,5.831000e+05,2.389000e+05,1.954100e+04
max,1984.000000,6663.300000,460305.000000,1.336790e+08,3.172035e+08,1.283890e+08,4.654194e+06


### Observations:
- The dataset contains information on crop production, farm value, and other metrics.
- Key columns include `Year`, `Country`, `Type of crop`, `Production (metric tonnes)`, and `Total farm value (dollars)`.
- There are no missing values after preprocessing.

## 2. Number of Crops Over the Years
We start by visualizing the number of crops recorded each year to identify any anomalies or trends.

In [ ]:
df['Year'].value_counts().sort_index().plot(kind='bar', figsize=(13, 5), color='skyblue')
plt.title('Number of Crops per Year')
plt.xlabel('Year')
plt.ylabel('Number of Crops')
plt.show()

### Observations:
- The year 1954 appears anomalous, as it includes data for 1955. This issue has been corrected by splitting the data based on the `Country` column.

In [ ]:
df.isnull().sum()
df.fillna(0, inplace=True)
df.rename(columns={'REF_DATE':'Year'}, inplace=True)
df.rename(columns={'GEO':'Country'},  inplace=True)
df= df[df['Type of crop'] != '0']
df.drop(df[df['Year'] == 1954].index, inplace=True)
df.to_csv('farm_corrected.csv', index=False)

In [ ]:
df=pd.read_csv('farm_corrected.csv')

## 3. Average Production by Crop Type
Next, we calculate the average production for each crop type to identify the highest and lowest producers.

In [ ]:
crop_production = df.groupby('Type of crop')['Production (metric tonnes)'].mean().sort_values(ascending=False)
crop_production_df = crop_production.reset_index()
crop_production_df['Production (metric tonnes)'] = crop_production_df['Production (metric tonnes)'] / 1e6  # Convert to millions

plt.figure(figsize=(10, 6))
sns.barplot(data=crop_production_df, y='Type of crop', x='Production (metric tonnes)', palette='viridis')
plt.xlabel("Average Production (in millions of metric tonnes)")
plt.title("Average Production by Crop Type")
plt.show()

### Observations:
- **Tame Hay** is the highest producer on average.
- **Peas, Dry** is the lowest producer on average.
- We will now focus on comparing these two crops in detail.

## 4. Yearly Trends for Tame Hay and Peas, Dry
We analyze the yearly production trends for the highest and lowest producers.

In [ ]:
# Tame Hay
tame_hay_yearly = df[df['Type of crop'] == 'Tame hay'].groupby('Year')['Production (metric tonnes)'].mean().reset_index()
plt.figure(figsize=(10, 6))
sns.lineplot(data=tame_hay_yearly, x='Year', y='Production (metric tonnes)', label='Tame Hay', color='green')
plt.title('Yearly Production of Tame Hay')
plt.xlabel('Year')
plt.ylabel('Production (metric tonnes)')
plt.show()

# Peas, Dry
peas_dry_yearly = df[df['Type of crop'] == 'Peas, dry'].groupby('Year')['Production (metric tonnes)'].mean().reset_index()
plt.figure(figsize=(10, 6))
sns.lineplot(data=peas_dry_yearly, x='Year', y='Production (metric tonnes)', label='Peas, Dry', color='orange')
plt.title('Yearly Production of Peas, Dry')
plt.xlabel('Year')
plt.ylabel('Production (metric tonnes)')
plt.show()

### Observations:
- Tame Hay shows consistent production over the years with some fluctuations.
- Peas, Dry has much lower production and shows more variability.

## 5. Country-Level Contributions
We analyze how different countries contribute to the production of Tame Hay and Peas, Dry.

In [ ]:
# Tame Hay by Country
tame_hay_by_country = df[df['Type of crop'] == 'Tame hay'].groupby('Country')['Production (metric tonnes)'].mean().sort_values(ascending=False)
tame_hay_by_country.plot(kind='bar', figsize=(10, 6), color='green')
plt.title('Average Production of Tame Hay by Country')
plt.xlabel('Country')
plt.ylabel('Production (metric tonnes)')
plt.show()

# Peas, Dry by Country
peas_dry_by_country = df[df['Type of crop'] == 'Peas, dry'].groupby('Country')['Production (metric tonnes)'].mean().sort_values(ascending=False)
peas_dry_by_country.plot(kind='bar', figsize=(10, 6), color='orange')
plt.title('Average Production of Peas, Dry by Country')
plt.xlabel('Country')
plt.ylabel('Production (metric tonnes)')
plt.show()

### Observations:
- Certain countries dominate the production of Tame Hay.
- Peas, Dry production is more evenly distributed but remains low overall.

## 6. Correlation Between Production and Farm Value
We explore the relationship between production and total farm value for all crops.

In [ ]:
sns.scatterplot(data=df, x='Production (metric tonnes)', y='Total farm value (dollars)', hue='Type of crop', alpha=0.7)
plt.title('Correlation Between Production and Total Farm Value')
plt.xlabel('Production (metric tonnes)')
plt.ylabel('Total Farm Value (dollars)')
plt.show()

### Observations:
- There is a positive correlation between production and farm value, as expected.
- Tame Hay contributes significantly to farm value, while Peas, Dry has a minimal impact.

## 7. Variability in Production
Finally, we analyze the variability in production for all crops to understand consistency.

In [ ]:
crop_variability = df.groupby('Type of crop')['Production (metric tonnes)'].std().sort_values(ascending=False)
crop_variability.plot(kind='bar', figsize=(12, 6), color='purple')
plt.title('Yearly Variability in Crop Production')
plt.xlabel('Type of Crop')
plt.ylabel('Standard Deviation of Production (metric tonnes)')
plt.show()

### Observations:
- Tame Hay shows moderate variability, indicating consistent production.
- Peas, Dry has higher variability relative to its low production levels.